In [22]:
from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq
import os

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    groq_api_key = os.environ['GROQ_API_KEY']
)

In [23]:
response = llm.invoke("who is the first person to land on the moon?")
print(response.content)

The first person to land on the moon was Neil Armstrong.  He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission.


In [5]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-36945")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Bewirb dich für Lead Cyber Defense Incident Responder

JobsucheNavigation überspringenJobsucheNIKE, INC. JOBSBefristete JobsJoin The Talent CommunityUnser Leben bei NikeÜbersichtLeistungenMarkenÜbersichtJordanConverseTeamsÜbersichtAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public Affairs
Human ResourcesData & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyStandorteÜbersichtNike WHQNike New York HQEHQ: Hilversum, NiederlandeELC: Laakdal, BelgiumGreater China HQDiversität, Gleichstellung und InklusionÜbersichtMilitärBehinderung und InklusionInklusion der indigenen BevölkerungPraktikaTechnologyLead Cyber Defense Incident ResponderShangh

In [6]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)


```json
{
  "role": "Lead Cyber Defense Incident Responder",
  "experience": "Minimum five years of security monitoring experience and incident response activities; preferably within a professional services firm or similar environment",
  "skills": [
    "Solid knowledge of information security principles and practice",
    "Strong knowledge of incident response and crisis management",
    "Cloud security knowledge and skills",
    "Log (network, security, access, OS, application, etc.) analysis skills",
    "Proficiency with Windows, Mac OSX and Linux",
    "Understanding of application security and dev-sec-ops",
    "Coding/scripting experience (PowerShell, Python, Ruby, Java, VB Scripting etc.)",
    "Malware analysis, virus exploitation and mitigation techniques experience"
  ],
  "description": "The Lead Cyber Defense Incident Responder will be responsible for identifying, containing and eradicating cyber security incidents. This position uses a wide range of tools and experience 

In [7]:
type(res.content)

str

In [8]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Lead Cyber Defense Incident Responder',
 'experience': 'Minimum five years of security monitoring experience and incident response activities; preferably within a professional services firm or similar environment',
 'skills': ['Solid knowledge of information security principles and practice',
  'Strong knowledge of incident response and crisis management',
  'Cloud security knowledge and skills',
  'Log (network, security, access, OS, application, etc.) analysis skills',
  'Proficiency with Windows, Mac OSX and Linux',
  'Understanding of application security and dev-sec-ops',
  'Coding/scripting experience (PowerShell, Python, Ruby, Java, VB Scripting etc.)',
  'Malware analysis, virus exploitation and mitigation techniques experience'],
 'description': 'The Lead Cyber Defense Incident Responder will be responsible for identifying, containing and eradicating cyber security incidents. This position uses a wide range of tools and experience to perform the incident response fun

In [9]:
type(json_res)

dict

In [13]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")

In [15]:
import chromadb
chroma_client = chromadb.PersistentClient('vectorstore')
collection = chroma_client.get_or_create_collection(name="portfolio")

In [16]:
import uuid

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"], metadatas={"links":row["Links"]}, ids = [str(uuid.uuid4())])

In [17]:
job = json_res
job['skills']

['Solid knowledge of information security principles and practice',
 'Strong knowledge of incident response and crisis management',
 'Cloud security knowledge and skills',
 'Log (network, security, access, OS, application, etc.) analysis skills',
 'Proficiency with Windows, Mac OSX and Linux',
 'Understanding of application security and dev-sec-ops',
 'Coding/scripting experience (PowerShell, Python, Ruby, Java, VB Scripting etc.)',
 'Malware analysis, virus exploitation and mitigation techniques experience']

In [18]:
links = collection.query(
    query_texts=job['skills'], # Chroma will embed this for you
    n_results=2 # how many results to return #ecuclidien distance
).get('metadatas',[])
links

[[{'links': 'https://example.com/ios-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/vue-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}]]

In [19]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Stefy, a business development executive at ABC. ABC is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of ABC 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase ABC's portfolio: {link_list}
        Remember you are Stefy, BDE at ABC. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Cyber Defense Incident Response Solutions for Your Organization

Dear Hiring Manager,

I came across the job posting for a Lead Cyber Defense Incident Responder and was impressed by the organization's commitment to robust cybersecurity measures. As a Business Development Executive at ABC, I'd like to introduce our company as a trusted partner in providing top-notch incident response solutions.

Our team at ABC has extensive experience in developing and implementing cutting-edge cybersecurity solutions, leveraging the latest technologies to ensure the security and integrity of our clients' systems. With a strong focus on cloud security, application security, and Unix/Linux operating systems, we're confident that our expertise aligns with your organization's needs.

Our portfolio showcases our capabilities in:

* DevOps solutions, ensuring seamless integration and automation of security processes (https://example.com/devops-portfolio)
* Machine Learning and Python develop